# Data Preprocessing

Notebook to load raw CSVs, clean and merge them, and save a single processed file `data/processed_data.csv`.

Notes:
- Looks for raw files in `data/rawdata/`, `data/raw/`, `rawdata/`, or `archive/`.
- Uses `src.preprocessing` utilities (`clean_numeric_columns`, `parse_dates`, `save_processed`) for consistent cleaning.


In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Ensure project root is on sys.path so `src` imports work when running cells
p = Path.cwd()
for _ in range(6):
    if (p / 'src').exists():
        sys.path.insert(0, str(p))
        break
    p = p.parent
else:
    # fallback: add current working dir
    sys.path.insert(0, str(Path.cwd()))

# Reuse preprocessing utilities from src for consistent behavior
from src.preprocessing import clean_numeric_columns, parse_dates, save_processed

## 1. Cấu hình và Đường dẫn
Khai báo đường dẫn đến các file dữ liệu.

In [24]:
from pathlib import Path

# Enforce canonical raw directory 'data/rawdata' and required filenames
p = Path.cwd()
base_path = None
for _ in range(6):
    candidate = p / 'data' / 'rawdata'
    if candidate.exists():
        base_path = candidate
        break
    p = p.parent

if base_path is None:
    raise FileNotFoundError("Missing required raw directory 'data/rawdata' (searching upward from notebook). Please place raw files there.")

required_files = ['Orders.csv', 'Order_Details.csv', 'Customers.csv']
missing = [f for f in required_files if not (base_path / f).exists()]
if missing:
    raise FileNotFoundError(f"Missing required raw files in {base_path}: {missing}. Please place raw files into 'data/rawdata' with names: {required_files}.")

orders_path = base_path / 'Orders.csv'
details_path = base_path / 'Order_Details.csv'
customers_path = base_path / 'Customers.csv'

print(f"Using raw data directory: {base_path}")

Using raw data directory: /Users/admin/ML/3A_Superstore/data/rawdata


## 2. Đọc dữ liệu
Đọc dữ liệu từ file CSV

In [25]:
print("Loading data...")

from src.preprocessing import robust_read_processed

# If raw split files exist, read them; otherwise try loading an existing processed file
if orders_path.exists() and details_path.exists() and customers_path.exists():
    # Robust CSV reader for raw split files
    def robust_read_csv(path, **kwargs):
        try:
            return pd.read_csv(path, **kwargs)
        except Exception:
            for sep in [';', ',']:
                for enc in ['utf-8', 'latin-1']:
                    try:
                        return pd.read_csv(path, sep=sep, encoding=enc, engine='python')
                    except Exception:
                        continue
            raise

    # Load Orders
    orders = robust_read_csv(orders_path)
    print(f"Orders shape: {orders.shape}")

    # Load Details
    details = robust_read_csv(details_path)
    print(f"Details shape: {details.shape}")

    # Load Customers
    customers = robust_read_csv(customers_path)
    print(f"Customers shape: {customers.shape}")
else:
    print("Raw split files not found. Attempting to load existing processed dataset 'data/processed_data.csv'...")

    # Try to find processed_data.csv by searching upward from current working directory
    p = Path.cwd()
    processed_path = None
    for _ in range(6):
        candidate = p / 'data' / 'processed_data.csv'
        if candidate.exists():
            processed_path = str(candidate)
            break
        candidate2 = p / 'processed_data.csv'
        if candidate2.exists():
            processed_path = str(candidate2)
            break
        p = p.parent

    if processed_path is None:
        # fallback to default behavior of robust_read_processed which searches local paths
        final_df, sep = robust_read_processed()
    else:
        final_df, sep = robust_read_processed(processed_path)

    print(f"Loaded processed dataset with shape: {final_df.shape}")

Loading data...
Orders shape: (10235193, 6)
Details shape: (51185032, 7)
Customers shape: (99998, 11)


## 3. Lấy mẫu (Sampling)
Giảm kích thước dữ liệu để thuận tiện cho việc xử lý và phân tích. Chúng ta lấy mẫu theo Orders ID để đảm bảo tính toàn vẹn của đơn hàng.

In [18]:
print("Sampling orders to reduce dataset size...")

if 'final_df' in globals():
    print("Processed dataset already loaded; skipping raw-order sampling.")
else:
    # If dataset is large, sample orders to keep processing fast for prototyping
    TARGET_ORDERS = 12000
    if len(orders) > TARGET_ORDERS:
        orders = orders.sample(n=TARGET_ORDERS, random_state=42)

    print(f"Sampled Orders shape: {orders.shape}")

Sampling orders to reduce dataset size...
Processed dataset already loaded; skipping raw-order sampling.


## 4. Làm sạch dữ liệu (Data Cleaning)
- Chuyển đổi định dạng tiền tệ (chuỗi có dấu phẩy sang float)
- Chuyển đổi định dạng ngày tháng
- Lọc dữ liệu chi tiết và khách hàng theo danh sách đơn hàng đã lấy mẫu

In [19]:
print("Cleaning data using src.preprocessing helpers...")

from src.preprocessing import clean_numeric_columns, parse_dates

if 'final_df' in globals():
    # Assume final_df already cleaned in previous run; ensure numeric normalization and date parsing
    final_df = clean_numeric_columns(final_df, cols=['TOTALBASKET','UNITPRICE','TOTALPRICE','AMOUNT'])
    final_df = parse_dates(final_df, col='DATE_')
    final_df = parse_dates(final_df, col='USERBIRTHDATE')
    print('Processed dataset cleaned (normalized numeric columns and parsed dates).')
else:
    # Clean Orders
    orders = clean_numeric_columns(orders, cols=['TOTALBASKET'])
    orders = parse_dates(orders, col='DATE_')

    # Filter and clean Details
    details = details[details['ORDERID'].isin(orders['ORDERID'])]
    details = clean_numeric_columns(details, cols=['UNITPRICE', 'TOTALPRICE'])

    # Filter and parse Customers
    customers = customers[customers['USERID'].isin(orders['USERID'])]
    customers = parse_dates(customers, col='USERBIRTHDATE')

    # Basic validation
    required = ['ORDERID','USERID']
    for dfname, df in [('orders', orders), ('details', details), ('customers', customers)]:
        for col in required:
            if col not in df.columns:
                raise KeyError(f"Required column {col} missing in {dfname} dataframe")

    # Show concise summaries
    print('Orders:', orders.shape)
    print('Details:', details.shape)
    print('Customers:', customers.shape)

Cleaning data using src.preprocessing helpers...


/Users/admin/ML/3A_Superstore/src/preprocessing.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
/Users/admin/ML/3A_Superstore/src/preprocessing.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')


Processed dataset cleaned (normalized numeric columns and parsed dates).


## 5. Hợp nhất dữ liệu (Merging)
Kết nối 3 bảng Orders, Order Details, và Customers lại thành một bảng duy nhất.

In [20]:
print("Merging data...")

if 'final_df' in globals():
    print(f"Skipping merge, using existing processed data with shape: {final_df.shape}")
else:
    # Merge Orders with Details on ORDERID
    merged_orders = pd.merge(details, orders, on='ORDERID', how='left')
    # Merge with Customers on USERID
    final_df = pd.merge(merged_orders, customers, on='USERID', how='left')

    print(f"Merged shape: {final_df.shape}")

    # If extremely large (unlikely after sampling), downsample for prototyping
    if len(final_df) > 1000000:
        final_df = final_df.sample(n=1000000, random_state=42)

    print('Merged and limited (if applicable):', final_df.shape)

Merging data...
Skipping merge, using existing processed data with shape: (50000, 25)


## 6. Feature Engineering
Tạo các đặc trưng mới:
- **Year, Month**: Từ ngày đặt hàng
- **Age**: Tuổi của khách hàng

In [21]:
# Ensure DATE_ is datetime and create time features
final_df['Year'] = final_df['DATE_'].dt.year
final_df['Month'] = final_df['DATE_'].dt.month

# Calculate Age from USERBIRTHDATE if available
def compute_age(birthdate):
    if pd.isna(birthdate):
        return pd.NA
    try:
        return pd.Timestamp.now().year - birthdate.year
    except Exception:
        return pd.NA

final_df['Age'] = final_df['USERBIRTHDATE'].apply(compute_age)

# Quick check
print('Columns now include:', ['Year','Month','Age'])
print('Sample rows:')
final_df[['Year','Month','Age']].head()

Columns now include: ['Year', 'Month', 'Age']
Sample rows:


,Year,Month,Age
0,2023,5,44
1,2023,6,37
2,2021,6,48
3,2021,12,-44
4,2022,2,36


## 7. Lưu và Kiểm tra kết quả
Lưu dữ liệu đã xử lý ra file CSV mới và hiển thị thông tin mẫu.

In [22]:
# Save processed data using utility to ensure consistent path/dirs
output_path = Path('data') / 'processed_data.csv'
save_processed(final_df, out=str(output_path))
print(f"Saved processed data to {output_path}")

# Quick inspection
print(final_df.info())
final_df.head()

Saved processed data to data/processed_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ORDERID        50000 non-null  int64         
 1   ORDERDETAILID  50000 non-null  int64         
 2   AMOUNT         50000 non-null  int64         
 3   UNITPRICE      50000 non-null  float64       
 4   TOTALPRICE     50000 non-null  float64       
 5   ITEMID         50000 non-null  int64         
 6   ITEMCODE       50000 non-null  int64         
 7   BRANCH_ID      50000 non-null  object        
 8   DATE_          50000 non-null  datetime64[ns]
 9   USERID         50000 non-null  int64         
 10  NAMESURNAME_x  50000 non-null  object        
 11  TOTALBASKET    50000 non-null  float64       
 12  USERNAME_      50000 non-null  object        
 13  NAMESURNAME_y  50000 non-null  object        
 14  STATUS_        50000 n

,ORDERID,ORDERDETAILID,AMOUNT,UNITPRICE,TOTALPRICE,ITEMID,ITEMCODE,BRANCH_ID,DATE_,USERID,...,USERGENDER,USERBIRTHDATE,REGION,CITY,TOWN,DISTRICT,ADDRESSTEXT,Year,Month,Age
0,6420952,32112344,4,95.40,422.36,10296,11170,33-AF2,2023-05-17,54220,...,E,1981-11-17,Ege,Afyonkarahisar,BOLVAD_N,KONAK MAH.,KONAK MAH. GLSU SOKAK 03300 BOLVAD_N/AFYONK...,2023,5,44
1,7219622,36109907,3,15.00,49.05,5123,25003,146-KA3,2023-06-12,84912,...,K,1988-01-24,Akdeniz,Kahramanmara_,KAHRAMANMARA_ MERKEZ,_EH_T ABDULLAH AVU_ MAH.,_EH_T ABDULLAH AVU_ MAH. 66005. SOKAK KAHR...,2023,6,37
2,4159842,20801845,3,7.85,14.22,5492,6167,421-DI2,2021-06-17,76879,...,E,1977-09-17,Gneydo_u Anadolu,Diyarbakõr,HAN_,ARDAKLI K Y,ARDAKLI K Y MAH. K YN KEND_S_ 21800 HAN_/...,2021,6,48
3,7594409,37983589,4,8.70,28.16,1264,2787,472-BA1,2021-12-20,61129,...,K,2069-03-07,Gneydo_u Anadolu,Batman,GERC_,BA_LARBA_I MAH.,BA_LARBA_I MAH. 118. SOKAK 72300 GERC_/BATMAN,2021,12,-44
4,7283856,36430669,3,57.25,164.34,16277,29876,427-GA3,2022-02-28,25971,...,K,1989-08-17,Gneydo_u Anadolu,Gaziantep,_EH_TKAM_L,8 _UBAT MAH.,8 _UBAT MAH. 82046 NOLU SOKAK 27500 _EH_TKAM...,2022,2,36
